In [ ]:
 # !curl -s https://course.fast.ai/setup/colab | bash
 !pip install git+https://github.com/fastai/fastai.git

  Cloning https://github.com/fastai/fastai.git to /tmp/pip-req-build-82bb5skt
  Running command git clone -q https://github.com/fastai/fastai.git /tmp/pip-req-build-82bb5skt
     |████████████████████████████████| 61kB 5.4MB/s 
  Created wheel for fastai: filename=fastai-2.1.10-cp36-none-any.whl size=188986 sha256=c595fd49afb1187ecf0594ab4dca5891d34c74e733d585cc88df086e204ae6c2
  Stored in directory: /tmp/pip-ephem-wheel-cache-tugz_x59/wheels/cf/46/39/b2d08762125ed2376861976ab2c4ac30c029b86e375735d9b8
Successfully built fastai
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [ ]:
from fastai.text.all import * 
import pandas as pd

In [ ]:
############################

In [ ]:
path = untar_data(URLs.IMDB)
get_imdb = partial(get_text_files, folders=['train','test','unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [ ]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj apparently the writer and director of this direct - to - dvd slasher movie is a fan of xxmaj friday 13th and other summer camp slashers . xxmaj this movie has everything - a group of teenagers who want to spend the weekend with fun , alcohol and sex in an abandoned summer camp called "" camp xxmaj blood "" , the old man who warns them not to go there ; and of course the crazy killer","xxmaj apparently the writer and director of this direct - to - dvd slasher movie is a fan of xxmaj friday 13th and other summer camp slashers . xxmaj this movie has everything - a group of teenagers who want to spend the weekend with fun , alcohol and sex in an abandoned summer camp called "" camp xxmaj blood "" , the old man who warns them not to go there ; and of course the crazy killer with"
1,"'m not even going to describe it . xxmaj you 'll know it when you hear it . xxmaj and you 'll agree with me . \n\n xxmaj there are some scenes of violence , sure . xxmaj and there is a xxmaj benny xxmaj hill style chase sequence involving a transsexual . xxmaj there is even an immortal bit of dialogue , that may or may not have been taken from xxmaj shakespeare or xxmaj john xxmaj milton ,","not even going to describe it . xxmaj you 'll know it when you hear it . xxmaj and you 'll agree with me . \n\n xxmaj there are some scenes of violence , sure . xxmaj and there is a xxmaj benny xxmaj hill style chase sequence involving a transsexual . xxmaj there is even an immortal bit of dialogue , that may or may not have been taken from xxmaj shakespeare or xxmaj john xxmaj milton , where"


In [ ]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3,
    metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.116470,3.914021,0.298929,50.100025,21:50


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'fastai-v/'

Mounted at /content/gdrive


In [ ]:
dest=Path(base_dir + "models/IMDBfinetuneLM/")
try:
  dest.mkdir(parents=True, exist_ok=False)
except FileExistsError:
  print ('File Already Exists')

File Already Exists


In [ ]:
learn.save(dest/'1epoch')

Path('/content/gdrive/My Drive/fastai-v/models/IMDBfinetuneLM/1epoch.pth')

In [ ]:
learn.load(dest/'1epoch');

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, 2e-3)
learn.save(dest/'3epoch')

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.959766,3.798783,0.320105,44.646820,22:49
1,3.813104,3.701815,0.330976,40.520798,22:30


Path('/content/gdrive/My Drive/fastai-v/models/IMDBfinetuneLM/3epoch.pth')

In [ ]:
learn.load(dest/'3epoch');
learn.unfreeze()
learn.fit_one_cycle(2, 2e-3)
learn.save(dest/'5epoch')

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.783722,3.689527,0.330082,40.025913,22:32
1,3.674037,3.626731,0.337466,37.589741,22:27


Path('/content/gdrive/My Drive/fastai-v/models/IMDBfinetuneLM/5epoch.pth')

In [ ]:
learn.load(dest/'5epoch');
learn.unfreeze()
learn.fit_one_cycle(2, 2e-3)
learn.save(dest/'7epoch')

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.699222,3.643095,0.334134,38.209923,22:29
1,3.610433,3.593846,0.340305,36.373703,22:55


Path('/content/gdrive/My Drive/fastai-v/models/IMDBfinetuneLM/7epoch.pth')

In [ ]:
learn.load(dest/'7epoch');
learn.unfreeze()
learn.fit_one_cycle(2, 2e-3)
learn.save(dest/'9epoch')

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.642051,3.618816,0.336316,37.293392,22:57
1,3.539463,3.577997,0.341511,35.801758,22:57


Path('/content/gdrive/My Drive/fastai-v/models/IMDBfinetuneLM/9epoch.pth')

In [ ]:
learn.load(dest/'9epoch');
learn.unfreeze()
learn.fit_one_cycle(2, 2e-3)
learn.save(dest/'11epoch')

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.605504,3.606565,0.337198,36.839302,23:08
1,3.507205,3.570506,0.342169,35.534573,22:42


Path('/content/gdrive/My Drive/fastai-v/models/IMDBfinetuneLM/11epoch.pth')

In [ ]:
learn.save_encoder(dest/'finetuned')

In [ ]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [ ]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos xxmaj by now you 've probably heard a bit about the new xxmaj disney dub of xxmaj miyazaki 's classic film , xxmaj laputa : xxmaj castle xxmaj in xxmaj the xxmaj sky . xxmaj during late summer of 1998 , xxmaj disney released "" kiki 's xxmaj delivery xxmaj service "" on video which included a preview of the xxmaj laputa dub saying it was due out in "" 1 xxrep 3 9 "" . xxmaj it 's obviously way past that year now , but the dub has been finally completed . xxmaj and it 's not "" laputa : xxmaj castle xxmaj in xxmaj the xxmaj sky "" , just "" castle xxmaj in xxmaj the xxmaj sky "" for the dub , since xxmaj laputa is not such a nice word in xxmaj spanish ( even though they use the word xxmaj laputa many times",pos
2,"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the sweetest and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that tries too hard , nor does it come up with the oddest possible scenarios to get the two protagonists together in the end . xxmaj in fact , all its charm is innate , contained within the characters and the setting and the plot … which is highly believable to boot . xxmaj it 's easy to think that such a love story , as beautiful as any other ever told , * could * happen to you … a feeling you do n't often get from other romantic comedies",pos


In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5,
                                metrics=accuracy).to_fp16()

In [ ]:
learn = learn.load_encoder(dest/'finetuned')

In [ ]:
#finetuning the classifier
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.336966,0.183626,0.930680,01:12


In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.253788,0.166638,0.936600,01:18


In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.205153,0.153461,0.942240,01:34


In [ ]:
learn.unfreeze
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.182732,0.150122,0.944240,01:34
1,0.173729,0.150698,0.944480,01:35


https://docs.fast.ai/callback.schedule.html#Learner.fit_one_cycle
https://towardsdatascience.com/transfer-learning-using-the-fastai-library-d686b238213e
